### Importar as Bibliotecas para rodar o código python OpenAI

In [35]:
#import OpenAI API and other necessary libraries

import os
from dotenv import load_dotenv
from scraper import fetch_website_contents
import requests
from IPython.display import display, Markdown
from openai import OpenAI




### Carregar o envorinment 

In [36]:
# Load environment variables
load_dotenv(override=True)
openai_api_key = os.getenv("OPENAI_API_KEY")

if not openai_api_key:
    display(Markdown("**Error:** OpenAI API key is missing. Please set the `OPENAI_API_KEY` environment variable."))

### Create Prompts for stocks

In [37]:
# Define system and user prompts for stock market analysis
system_prompt = """You are an expert stock market analyst with deep knowledge of financial markets, 
technical analysis, and fundamental analysis. Your role is to provide clear, data-driven insights 
about stocks, market trends, and investment opportunities. You analyze financial data objectively 
and present balanced perspectives on both risks and opportunities."""

user_prompt = """Please analyze the current stock market conditions and provide insights on the following look for Cotação de ITUB4 in the output:

1. Overall market sentiment and key trends
2. Company  performance 
3. Notable stock movements and the reasons behind them
4. Potential opportunities and risks for investors
5. Key technical indicators (graphs like moving averages, RSI, MACD and others) to watch for this company as better investment decisions

Please provide a comprehensive analysis based on recent market data."""

print("System Prompt:")
print(system_prompt)
print("\n" + "="*80 + "\n")
print("User Prompt:")
print(user_prompt)

System Prompt:
You are an expert stock market analyst with deep knowledge of financial markets, 
technical analysis, and fundamental analysis. Your role is to provide clear, data-driven insights 
about stocks, market trends, and investment opportunities. You analyze financial data objectively 
and present balanced perspectives on both risks and opportunities.


User Prompt:
Please analyze the current stock market conditions and provide insights on the following look for Cotação de ITUB4 in the output:

1. Overall market sentiment and key trends
2. Company  performance 
3. Notable stock movements and the reasons behind them
4. Potential opportunities and risks for investors
5. Key technical indicators (graphs like moving averages, RSI, MACD and others) to watch for this company as better investment decisions

Please provide a comprehensive analysis based on recent market data.


In [ ]:
# Define the URL to scrape market data from
url = "https://www.infomoney.com.br/cotacoes/b3/acao/itau-unibanco-itub4/"

# Fetch the website content
scraped_content = fetch_website_contents(url)   
    
print(f"Successfully scraped content from: {url}")
print(f"Content length: {len(scraped_content)} characters")
print(scraped_content)


# Display the response
def analyze_stock_with_ai(scraped_content):
    """
    Enhance prompt with scraped data, and get AI analysis.
    
    Args:
        scraped_content: The scraped HTML content from the website

    Returns:
        The AI response content
    """
    
    
    # Initialize OpenAI client
    client = OpenAI(api_key=openai_api_key)
    
    # Enhance user prompt with scraped market data
    enhanced_prompt = f"""
    Based on the following real-time market data from {scraped_content}:

    {scraped_content}

    {user_prompt}
    """
    
    # Get AI response
    response = client.chat.completions.create(
        model="gpt-5-nano",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": enhanced_prompt}
        ]
    )

    return response.choices[0].message.content

# Call the function and display result
result = analyze_stock_with_ai(url)
display(Markdown(result))




In [ ]:
%pip install fundamentus

import fundamentus

# Fetch fundamental data from Fundamentus API
fundamentus_ticker = "ITUB4"
fundamentus_url = f"https://fundamentus.com.br/api/papel/{fundamentus_ticker}"

try:
    fundamentus_response = requests.get(fundamentus_url)
    fundamentus_data = fundamentus_response.json() if fundamentus_response.status_code == 200 else {}
    
    print(f"Fundamentus data for {fundamentus_ticker}:")
    print(fundamentus_data)
    
    # Combine scraped content with fundamentals
    combined_prompt = f"""
    Based on the following market data and fundamental indicators:
    
    
    **Market Data from InfoMoney:**
    {scraped_content}
    
    **Fundamental Data from Fundamentus:**
    {fundamentus_data}
    
    {user_prompt}
    """
    
    # Get AI analysis with fundamentals
    client = OpenAI(api_key=openai_api_key)
    
    response = client.chat.completions.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": combined_prompt}
        ]
    )
    
    analysis_with_fundamentals = response.choices[0].message.content
    display(Markdown("## Análise com Dados Fundamentalistas"))
    display(Markdown(analysis_with_fundamentals))
    
except Exception as e:
    print(f"Error fetching Fundamentus data: {e}")
    display(Markdown(f"**Error:** Could not fetch fundamental data - {e}"))

/Users/felipesba/Documents/Apps/Udemy/llm_engineering/.venv/bin/python: No module named pip
Note: you may need to restart the kernel to use updated packages.


ModuleNotFoundError: No module named 'fundamentus'